In [1]:
import tensorflow as tf
import scipy.io as scio
import scipy.sparse as sp
import pandas as pd
import numpy as np

In [2]:
##################33
#关闭GPU
import os
os.environ["CUDA_VISIBLE_DEVICES"]="-1"    


In [3]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


False

In [4]:
from tensorflow.keras import activations, regularizers, constraints, initializers

class Multi_GCNConv(tf.keras.layers.Layer):
    def __init__(self,
                 units,
                 activation=lambda x: x,
                 kernel_initializer='glorot_uniform',
                 bias_initializer='zeros',
                 **kwargs):
        super(Multi_GCNConv, self).__init__()

        self.units = units
        self.activation = activations.get(activation)
        self.kernel_initializer = initializers.get(kernel_initializer)
        self.bias_initializer = initializers.get(bias_initializer)
    
  


    def build(self, input_shape):

        """ GCN has two inputs : [shape(AnCount,An), shape(X_count,X)]
        """

        fdim = input_shape[1][-1]  # feature dim,取特征长度
        # 初始化权重矩阵
    
        
        #改成根据输入的邻接矩阵数量，设置等量的weight和bias：
        weight_count=input_shape[0][0]
        
        
        self.weight=self.add_weight(name="weight",
                               shape=(weight_count,fdim, self.units),
                               initializer=self.kernel_initializer,
                               trainable=True)
       
            
        self.bias = self.add_weight(name="bias",
                               shape=(self.units ),
                               initializer=self.bias_initializer,
                               trainable=True)
      
            
            

    def call(self, inputs):
        """ GCN has two inputs : [An, X]
        """
        #读入的是多个邻接矩阵
        self.Multi_An=inputs[0]
        self.X = inputs[1]
        
        #输入的稀疏张量 转 密集张量，因为稀疏张量无法进行3维矩阵乘法
        self.Multi_An=tf.sparse.to_dense(self.Multi_An)
        
        # 计算 XW 

        h = tf.add(tf.matmul(self.X, self.weight),self.bias) #这一步可以用sparseTensor，不知道有没有必要
            
        # 计算 AXW ,A矩阵已经处理过的
        
        output = tf.matmul(self.Multi_An, h)

        #output = tf.sparse.sparse_dense_matmul(self.An, h)
     

        if self.activation:
            output = self.activation(output)

        return output

In [5]:
class Self_Attention(tf.keras.layers.Layer):
    def __init__(self,
                 att_size,
                 activation=lambda x: x,
                 kernel_initializer='glorot_uniform',
                 bias_initializer='zeros',):
        super(Self_Attention, self).__init__()
        self.activation = activations.get(activation)
        self.att_size=att_size
        self.kernel_initializer = initializers.get(kernel_initializer)
        self.bias_initializer = initializers.get(bias_initializer)
        
    def build(self, input_shape):
        
        weight_att_shape_x=input_shape[-1]
        weight_att_shape_y=self.att_size
        
        self.weight_att=self.add_weight(name="weight_to_att",
                               shape=(weight_att_shape_x, weight_att_shape_y),
                               initializer=self.kernel_initializer, 
                               trainable=True)
        
        self.bias=self.add_weight(name="bias",
                               shape=(weight_att_shape_y,),
                               initializer=self.bias_initializer,
                               trainable=True)
        
        self.weight_att_u=self.add_weight(name="weight_to_att_u",
                               shape=(weight_att_shape_y, 1),
                               initializer=self.kernel_initializer,
                               trainable=True)

    
    def call(self, inputs):

        self.X = inputs
        
        WX=tf.matmul(self.X,self.weight_att)
        V=tf.tanh(tf.add(WX,self.bias))
        VU=tf.matmul(V,self.weight_att_u)
        alphas=tf.nn.softmax(VU,axis=0) 
        
        outputs=tf.multiply(alphas,self.X)
        outputs_att=tf.reduce_sum(outputs,axis=0) 
        
#         print('inputs.shape=',inputs.shape)
#         print('VU.shape=',VU.shape)
#         print('alphas.shape=',alphas.shape)
        alphas_new=tf.transpose(tf.squeeze(alphas, axis=-1)) #需要改变attention形状

        return outputs_att,alphas_new
    

In [6]:
import tensorflow as tf
from tensorflow.keras import activations, regularizers, constraints, initializers

class Multi_GC_MC_Conv(tf.keras.layers.Layer):
    def __init__(self,
                 units,
                 activation=lambda x: x,
                 kernel_initializer='glorot_uniform',
                 bias_initializer='zeros',
                 **kwargs):
        super(Multi_GC_MC_Conv, self).__init__()

        self.units = units #要降成的维度
        self.activation = activations.get(activation)
        self.kernel_initializer = initializers.get(kernel_initializer)
        self.bias_initializer = initializers.get(bias_initializer)
    
  


    def build(self, input_shape):
        #print(input_shape)

        """
        GC-MC has three inputs : [shape(AnCount,An), shape(X_count,X),shape(Z_count,Z)]
        X is self_feature,Z is another node type feature
        
        """
        
        
        ###   目标为XW1+(CA)ZW2    ###
        
        
         
        ###   这里开始为设置XW1的参数
        
        
        fdim_X = input_shape[1][-1]  # feature dim,取X特征长度
        weight_count=input_shape[0][0]   #改成根据输入的邻接矩阵数量，设置等量的weight和bias：
        
        self.weight_1=self.add_weight(name="weight_1",
                               shape=(weight_count,fdim_X, self.units),
                               initializer=self.kernel_initializer,
                               trainable=True)
       
        self.bias_1 = self.add_weight(name="bias_1",
                               shape=(self.units ),
                               initializer=self.bias_initializer,
                               trainable=True)
        
        ###   这里设置(CA)ZW2的参数
        
        fdim_Z=input_shape[2][-1]
        
        self.weight_2=self.add_weight(name="weight_2",
                               shape=(weight_count,fdim_Z, self.units), #fdim_Z这个不对，不是12100，是5403
                               initializer=self.kernel_initializer,
                               trainable=True)
       
        self.bias_2 = self.add_weight(name="bias_2",
                               shape=(self.units ),
                               initializer=self.bias_initializer,
                               trainable=True)
      
            
            

    def call(self, inputs):
        """ GC-MC has three inputs : [An, X，Z]
        """
        self.Multi_An=inputs[0]
        self.X = inputs[1]
        self.Z = inputs[2]
        
        #输入的稀疏张量 转 密集张量，因为稀疏张量无法进行3维矩阵乘法
        self.Multi_An=tf.sparse.to_dense(self.Multi_An)
        
        ### 计算 XW1
        
        #print('self.X, self.weight_1',self.X,self.weight_1)

        XW1 = tf.add(tf.matmul(self.X, self.weight_1),self.bias_1) #这一步可以用sparseTensor，不知道有没有必要
        
                
        
        ### 计算 (CA)ZW2
        
        #计算C，不需要了，在数据预处理部分已经计算了CA
        pass
        
        #计算ZW2
 
       
        
        ZW2 = tf.add(tf.matmul(self.Z, self.weight_2),self.bias_2) #这一步可以用sparseTensor，不知道有没有必要
        
        
        
        # 计算 (CA)T ZW2

        
        if self.Multi_An.shape[-1]!=ZW2.shape[1]:
            
            CA_T=tf.transpose(self.Multi_An,perm=[0,2,1])
            #print('需要转置，CA_T.shape=',CA_T.shape)
            output1= tf.matmul(CA_T,ZW2) 
        else:

            output1= tf.matmul(self.Multi_An,ZW2)  


        
        #计算 XW1+(CA)T ZW2
      
        output=tf.add(XW1,output1)        
               
#         if self.use_bias:
#             output = tf.nn.bias_add(output, self.bias) 
        if self.activation:
            output = self.activation(output)

        return output

In [7]:
def preprocess_graph(adj):
    # _A = A + I
    _adj = adj + sp.eye(adj.shape[0])
    # _dseq：各个节点的度构成的列表
    _dseq = _adj.sum(1).A1
    # 构造开根号的度矩阵
    _D_half = sp.diags(np.power(_dseq, -0.5))
    # 计算标准化的邻接矩阵, @ 表示矩阵乘法
    adj_normalized = _D_half @ _adj @ _D_half
    return adj_normalized.tocsr()

In [8]:
# 稀疏矩阵转稀疏张量
def sp_matrix_to_sp_tensor(M):
    if not isinstance(M, sp.csr.csr_matrix):
        M = M.tocsr()
    # 获取非0元素坐标
    row, col = M.nonzero()
    # SparseTensor参数：二维坐标数组，数据，形状
    X = tf.SparseTensor(np.mat([row, col]).T, M.data, M.shape)
    X = tf.cast(X, tf.float32)
    return X

In [9]:
def preprocess_adj_matrix_to_sparse_3D(Paths_name,data):
    
    adj_matrix_list=[]
    for name in Paths_name:
        #mat=data.get(name)
        mat=data[name]
        
        #如果是方阵，令对角线元素为零, 处理成DAD
        if mat.shape[0]==mat.shape[1]:
            #mat[np.eye(mat.shape[0],dtype=bool)]=0
            mat=sp.csr_matrix(mat)
            mat=preprocess_graph(mat) #A to DAD
            
        else: #非方阵，处理成CA
            #统一矩阵形式，N x M, N>=m 
            if mat.shape[0] < mat.shape[1]:
                mat=np.transpose(mat)
                
            #计算C
            sum_row=np.sum(mat,axis=1)
            sum_column=np.sum(mat,axis=0)
            
            #sum_row=np.expand_dims(sum_row,axis=0)
            #sum_column=np.expand_dims(sum_column,axis=0)
            
            #sum_row=np.transpose(sum_row,axes=(1,0))
            
            
            C=sum_row @ sum_column
            
            #C矩阵预处理,
            #C[C==0]=1 
            C=1./np.sqrt(C) 
            C=np.nan_to_num(C,0) #将nan转成0
            #C=C.T
         
            
   
            #CA
            mat=mat.todense()
   
        
            mat=np.multiply(C,mat)
            
            mat=sp.csr_matrix(mat)
           
    
        mat=sp_matrix_to_sp_tensor(mat)       #稀疏矩阵 转 稀疏张量
        
        

        
        mat=tf.sparse.expand_dims(mat,axis=0) #2D 扩 3D
        

        adj_matrix_list.append(mat)
       
    sparse_matrix_3D=tf.sparse.concat(0,adj_matrix_list) #3D张量堆叠
    print('矩阵处理完毕')
    return sparse_matrix_3D

####################################

In [10]:
class Multi_GCN_GCMC_Model(tf.keras.Model):
    def __init__(self,
                 Muilt_MM,
                 Muilt_MA,
                 Muilt_MD,
             
             
                 
                 Muilt_FM,
                 Muilt_FA,
                 Muilt_FD,
           
    
                 output_size=3): 

        # 调用父类__init__()方法
        super(Multi_GCN_GCMC_Model, self).__init__()
        
        self.Muilt_MM=Muilt_MM
        self.Muilt_MA=Muilt_MA
        self.Muilt_MD=Muilt_MD
        
        
        
        
        self.Muilt_FM =Muilt_FM
        self.Muilt_FA =Muilt_FA
        self.Muilt_FD =Muilt_FD
      
     
        
        #同质图
        self.Multi_GCN_layer_1_for_MM = Multi_GCNConv(64,activation=tf.keras.activations.relu)
       

        
        #二部图
        self.Multi_GCMC_layer_1_for_MA = Multi_GC_MC_Conv(64,activation=tf.keras.activations.relu)
        self.Multi_GCMC_layer_1_for_MD = Multi_GC_MC_Conv(64,activation=tf.keras.activations.relu)
     
   
        
        
        #注意力
        self.att_layer = Self_Attention(32)
 
    
        #Decoder部分
      
        self.dense2=tf.keras.layers.Dense(output_size)





    def call(self,input_x_id, training=False,dropout=0.):
        # 输入数据
        
        #同质网络I 编码过程
   
        _h1_for_M = self.Multi_GCN_layer_1_for_MM([self.Muilt_MM, self.Muilt_FM])
        if training:
            _h1_for_M = tf.nn.dropout(_h1_for_M, dropout) 
        
  

         #二部图MA 编码过程
  
        _h2_for_M = self.Multi_GCMC_layer_1_for_MA([self.Muilt_MA,  self.Muilt_FM, self.Muilt_FA])
        
        if training:
            _h2_for_M = tf.nn.dropout(_h2_for_M, dropout)
         
        
        #二部图MD 编码过程
  
        _h3_for_M = self.Multi_GCMC_layer_1_for_MD([self.Muilt_MD,  self.Muilt_FM, self.Muilt_FD])
        
        if training:
            _h3_for_M = tf.nn.dropout(_h3_for_M, dropout)
            
  

        #注意力需要最后连接，同质图和二部图的表示向量，

        _h6_for_M = tf.concat([_h1_for_M,_h2_for_M,_h3_for_M],axis=0) 

        
        

        _h7_for_M,att_for_M=self.att_layer(_h6_for_M) 

        
        
    

        #解码层

        _h8_for_M=self.dense2(_h7_for_M) 

        
        
        
        
        #####################################################################
        output_list=[]
        for idx in input_x_id:
            output_list.append(_h8_for_M[idx]) #返回指定id的所有向量
        
        
            
        #####################################################################
            
        embedding=_h7_for_M

        #######################################################################
        att=[att_for_M,]

       
        return output_list,att,embedding

In [11]:
#获取文件夹下所有文件名
import os
path=r'C:\Users\Hone\Desktop\Experimental_code\IMDB\计算的各个元路径矩阵'
name_list=[]
filelist=os.listdir(path)
for filename in filelist:
    name_list.append(filename)

In [12]:
#取出矩阵的名字
matrix_name_list=[]
for name in name_list:
    matrix_name_list.append(name.split('.')[0].split('_matrix_')[1])

In [13]:
matrix_name_list

['AMDM', 'DM', 'MA', 'MAM', 'MAMA', 'MAMAM', 'MAMD', 'MAMDM', 'MDM']

In [14]:
def find_matrix_from_file(path,key_name):
    filelist=os.listdir(path)
    
    for name in filelist:
        if key_name==name.split('.')[0].split('_matrix_')[1]:
            Matrix = sp.load_npz(path+'\\'+name) #读取
            return Matrix
        else:
            pass
        

    print('cant find %s in %s' %key_name %path)
    

In [15]:
#需要将矩阵按照类型进行区分，放进不同的dict
data_MM={}
data_MA={}
data_MD={}




for pathname in matrix_name_list:
    if pathname[0]==pathname[-1]: #同质邻接矩阵
        data_MM[pathname]=find_matrix_from_file(path,pathname) #输入目标位置文件，与指定文件名，返回该稀疏矩阵
    elif pathname[0]=='A' or pathname[-1]=='A':
        data_MA[pathname]=find_matrix_from_file(path,pathname) 
    elif pathname[0]=='D' or pathname[-1]=='D':
        data_MD[pathname]=find_matrix_from_file(path,pathname)     

In [16]:
Paths_name_MM=[keyword for keyword in data_MM]
Paths_name_MA=[keyword for keyword in data_MA]
Paths_name_MD=[keyword for keyword in data_MD]

In [17]:
# Paths_name_MM=['MAM',  'MDM']
# Paths_name_MA=['AMDM','MA','MAMA']

# 下面是矩阵预处理过程，包括DAD,CA，再转成3维数据

In [18]:
#处理I矩阵，2维变3维,变sparseTensor
MM_matrix = preprocess_adj_matrix_to_sparse_3D(Paths_name_MM,data_MM) #输入的是list和dict数据
MA_matrix = preprocess_adj_matrix_to_sparse_3D(Paths_name_MA,data_MA)
MD_matrix = preprocess_adj_matrix_to_sparse_3D(Paths_name_MD,data_MD)

矩阵处理完毕
矩阵处理完毕
矩阵处理完毕


In [19]:
FM=np.eye(4278)
FA=np.eye(5257)
FD=np.eye(2081)



FM=np.expand_dims(FM,axis=0)
FA=np.expand_dims(FA,axis=0)
FD=np.expand_dims(FD,axis=0)

In [20]:
model=Multi_GCN_GCMC_Model(MM_matrix,
                           MA_matrix,
                           MD_matrix,
                       
                      
                           
                           FM,
                           FA,
                           FD,                         
                           output_size=3
                          )

In [21]:
#输入训练id集合
import random

index_list_for_FM=[i for i in range(4278)] #需要随机化
random.shuffle(index_list_for_FM) #index乱序
    
a_part=int(len(index_list_for_FM)/10)
train_idx_for_FM = index_list_for_FM[:int(a_part*2)]  # 2 1 7
val_idx_for_FM = index_list_for_FM[ int(a_part*2)+1:int(a_part*3)]
test_idx_for_FM = index_list_for_FM[ int(a_part*3)+1:len(index_list_for_FM)]

In [22]:
output,att,embedding=model(index_list_for_FM,training=True,dropout=0.)

In [23]:
model.summary()

Model: "multi_gcn_gcmc__model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 multi_gcn_conv (Multi_GCNCo  multiple                 1095232   
 nv)                                                             
                                                                 
 multi_gc_mc__conv (Multi_GC  multiple                 1830848   
 _MC_Conv)                                                       
                                                                 
 multi_gc_mc__conv_1 (Multi_  multiple                 814080    
 GC_MC_Conv)                                                     
                                                                 
 self__attention (Self_Atten  multiple                 2112      
 tion)                                                           
                                                                 
 dense (Dense)               multiple        

In [24]:
#标签
data_label=np.load(r'C:\Users\Hone\Desktop\Experimental_code\IMDB\movie_label_one_hot.npy') #读取

In [25]:
data_label.shape

(4278, 3)

In [26]:
optimizer=tf.keras.optimizers.Adam()#learning_rate=0.01
loss_func=tf.keras.losses.CategoricalCrossentropy(from_logits=True)
#这个loss函数还不够全，还得加入L2正则化


train_loss=tf.keras.metrics.Mean('train_loss') #计算loss的均值
train_acc_I=tf.keras.metrics.CategoricalAccuracy('train_accuracy_I')


test_loss=tf.keras.metrics.Mean('test_loss')
test_acc_I=tf.keras.metrics.CategoricalAccuracy('test_accuracy_I')


In [27]:
def train_step(model,id_x,labels_x,dropout=0.):  #一个set为一个batch的量，
    #tf.GradientTape()记录运算过程的变量梯度
    
    with tf.GradientTape() as t:
        output_list,att,embedding=model(input_x_id=id_x,training=True,dropout=dropout)    #计算一个batch的预测值   #!!!
        loss_step=loss_func(labels_x,output_list)
 
    grads=t.gradient(loss_step,model.trainable_variables)  #更新变量的梯度
    
    
    #利用跟新的梯度计算变量，model.trainable_variables表示网络中所有的变量
 
    optimizer.apply_gradients(zip(grads,model.trainable_variables))  
    train_loss(loss_step)  #传入计算一个epoch累计loss函数中
    train_acc_I(labels_x,output_list) #传入计算一个epoch累计accuracy函数中
    
    return att

In [28]:
def test_step(model,id_x,labels_x): 
    output_list,att,embedding=model(input_x_id=id_x,training=False,dropout=0.) 
    loss_step=loss_func(labels_x,output_list)
    
    test_loss(loss_step)
    test_acc_I(labels_x,output_list) #传入计算一个epoch累计accuracy函数中


In [29]:
def split_list_by_n(list_collection, n):
    """
    将集合均分，每份n个元素
    :param list_collection:
    :param n:
    :return:返回的结果为评分后的每份可迭代对象
    """
    for i in range(0, len(list_collection), n):
        yield list_collection[i: i + n]

In [30]:
def train(
          MM_matrix,
          MA_matrix,
          MD_matrix,
       
          FM,
          FA,
          FD,
      
          epoch_time,  
          batch_size,
          
          
          id_x_set, 
          id_x_set_for_val,
          id_x_set_for_test,
          
  
          label_set_I,
          output_size=3,
          dropout=0.
         ): #需要给要训练的集合和验证集合
    
    model=Multi_GCN_GCMC_Model(MM_matrix,
                               MA_matrix,
                               MD_matrix,
                            
                         
                               
                               FM,
                               FA,
                               FD,
                             
         
                               output_size=output_size
                               )

    
    best_acc_result=0
    for epoch in range(epoch_time):
        
        #training
        batch_idx_train=[] #获取一个batch的数据
        for i in split_list_by_n(id_x_set,batch_size): #split_list_by_n这个函数
            batch_idx_train.append(i)  
                     
        #创建batch_label矩阵

        
        for a_batch_x in batch_idx_train: 
            
            batch_label_list_x=[]

            for id_x in a_batch_x:
                batch_label_list_x.append(label_set_I[id_x]) 
                
    

            att=train_step(model,
                           id_x=a_batch_x,
                           labels_x=batch_label_list_x,
                           dropout=dropout) 
            print('一个train_batch训练完毕')
                           
        print('Epoch {} ，train_loss={:.3f}，train_acc_I={:.3f}'.format(epoch,
                                                                        train_loss.result(),
                                                                        train_acc_I.result())) 
        
        
        
        
        
        #validating
        batch_idx_val=[] #获取一个batch的数据
        
        
        
        for i in split_list_by_n(id_x_set_for_val,batch_size):
            batch_idx_val.append(i)
            

            
        for a_batch_x in batch_idx_val:
            batch_label_list_x=[]
            
            for id_x in a_batch_x:
                batch_label_list_x.append(label_set_I[id_x]) #获取一个batch的所有label

                
            
            test_step(model,id_x=a_batch_x, labels_x=batch_label_list_x)
            print('一个val_batch验证完毕')
            
        print('val_loss={:.3f}，val_accuracy_I={:.3f},best_accuracy={:.3f}'.format(test_loss.result(),test_acc_I.result(),best_acc_result)
             )
        
        ####################################################################################
        #保存结果最佳的embedding
        test_acc=float(test_acc_I.result())
        if test_acc>best_acc_result:
            
            best_acc_result=test_acc
           
            
            #这个阈值保存满足条件的embedding
            if best_acc_result>0.45:
                
                #需要返回，train数据载入得到的att
                att=train_step(model,
                    id_x=id_x_set,
                    labels_x=label_set_I[id_x_set]) 
    
                #输出test数据集的embedding 
                #输入test数据集，输出test数据集的embedding
                _,att_test,embedding=model(input_x_id=id_x_set_for_test,
                          training=False) 
                #还需要一个对应的label矩阵
                #从label_set_I中取出对应id的值，重新组成一个新的矩阵
                embedding_list=[]
                new_label_matrix_list=[]
                for i in id_x_set_for_test:
                    new_label_matrix_list.append(label_set_I[i])
                    embedding_list.append(embedding[i])
                    new_label_matrix=np.array(new_label_matrix_list)
    
                #保存embedding和对应的label矩阵
                np.save(r'C:\Users\Hone\Desktop\My_model-DATA\IMDB\embedding_of_test_data_GCN',np.array(embedding_list))
                np.save(r'C:\Users\Hone\Desktop\My_model-DATA\IMDB\label_matrix_of_test_data_GCN',new_label_matrix)
                print('表示向量保存完毕！')

        train_loss.reset_states()
        train_acc_I.reset_states()

        
        test_loss.reset_states()
        test_acc_I.reset_states()
        ####################################################################################
        
        
        
    #循环结束,对test数据验证
    batch_idx_test = []  # 获取一个batch的数据

    
    for i in split_list_by_n(id_x_set_for_test, batch_size):
        batch_idx_test.append(i)

    

    for a_batch_x in batch_idx_test:
        batch_label_list_x = []

        
        for id_x in a_batch_x:
            batch_label_list_x.append(label_set_I[id_x])  # 获取一个batch的所有label
    
        

        test_step(model,id_x=a_batch_x,labels_x=batch_label_list_x)
    print('End of model training\n'
          'test_loss={:.3f}，test_accuracy_I={:.3f}'.format(test_loss.result(),
                                                          test_acc_I.result()))
    

    test_loss.reset_states()
    test_acc_I.reset_states()
    
    
    return att

In [31]:
att=train(
    MM_matrix=MM_matrix,
    MA_matrix=MA_matrix,
    MD_matrix=MD_matrix,

 
    
    FM=FM,
    FA=FA,
    FD=FD,

   
    epoch_time=50,  
    batch_size=1000,
          
          
    id_x_set          =train_idx_for_FM, 
    id_x_set_for_val  =val_idx_for_FM,
    id_x_set_for_test =test_idx_for_FM,

  
    label_set_I=data_label,
    output_size=data_label.shape[1],
    dropout=0.5
)

一个train_batch训练完毕
Epoch 0 ，train_loss=1.098，train_acc_I=0.345
一个val_batch验证完毕
val_loss=1.097，val_accuracy_I=0.406,best_accuracy=0.000
一个train_batch训练完毕
Epoch 1 ，train_loss=1.096，train_acc_I=0.395
一个val_batch验证完毕
val_loss=1.097，val_accuracy_I=0.423,best_accuracy=0.406
一个train_batch训练完毕
Epoch 2 ，train_loss=1.094，train_acc_I=0.547
一个val_batch验证完毕
val_loss=1.096，val_accuracy_I=0.390,best_accuracy=0.423
一个train_batch训练完毕
Epoch 3 ，train_loss=1.090，train_acc_I=0.657
一个val_batch验证完毕
val_loss=1.096，val_accuracy_I=0.380,best_accuracy=0.423
一个train_batch训练完毕
Epoch 4 ，train_loss=1.087，train_acc_I=0.683
一个val_batch验证完毕
val_loss=1.095，val_accuracy_I=0.373,best_accuracy=0.423
一个train_batch训练完毕
Epoch 5 ，train_loss=1.084，train_acc_I=0.692
一个val_batch验证完毕
val_loss=1.094，val_accuracy_I=0.390,best_accuracy=0.423
一个train_batch训练完毕
Epoch 6 ，train_loss=1.080，train_acc_I=0.701
一个val_batch验证完毕
val_loss=1.094，val_accuracy_I=0.387,best_accuracy=0.423
一个train_batch训练完毕
Epoch 7 ，train_loss=1.077，train_acc_I=0.699


In [32]:
att_1=tf.reduce_sum(att[0],axis=0)/att[0].shape[0]
att_weihgt_1=list(np.array(att_1))
for i in att_weihgt_1:
    print('%.3f' %i)
    
Paths_name_MM

0.013
0.011
0.011
0.014
0.277
0.276
0.253
0.076
0.070


['MAM', 'MAMAM', 'MAMDM', 'MDM']

In [33]:
AVG_ATT=np.mean(att_weihgt_1)
AVG_ATT

0.111111104